Tutorial : Jupyter를 위한 Docker 그리고 AWS
======================

# Jupyter Notebook

> The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and explanatory text. Uses include: data cleaning and transformation, numerical simulation, statistical modeling, machine learning and much more.
> http://jupyter.org

* IPython Notebook 을 계승.
* 프로젝트 초기에 기본적으로 제공했던 언어인 Julia, Python, R 을 합쳐 Jupyter 라는 이름을 만듦.

## Jupyter Notebook 맛보기

* [try.jupyter.org](https://try.jupyter.org)
  * 5분간 Jupyter Notebook 을 사용 해 볼 수 있는 공식 Example Server.

* [인하대학교 수학과 Jupyter Hub](https://jupyter.inha.ac.kr)
  * Intel(R) Xeon(R) CPU E5645 @ 2.40GHz (6-core, 2 CPUs), 1대.
  * Memory : 26G.
  * 인하대학교 수학과 학생 전용.

* [NIMS Jupyter Hub](https://jupyter.nims.re.kr)
  * Intel(R) Xeon(R) CPU E5-2630 v3 @ 2.40GHz (8-core, 2 CPUs), 계산서버 14대 + JupyterHub + Storage.
  * Memory : 64G.
  * 공개 (GitHub 인증).

### Kernel

* Jupyter Notebook 에서 프로그래밍 언어를 사용하기 위해서는 해당 언어를 위한 kernel 을 설치 해야합니다.

![Kernels](kernels.png)

### Notebook file

* Notebook 은 하나의 실행 단위인 Cell 들로 구성이 되어 있습니다.
* 각 Cell 에 Code 나 Markdown 을 작성할 수 있습니다.
![Cell](cell.png)
  * Heading 은 더이상 사용하지 않습니다 (Markdown 의 Heading 을 사용하세요).
  * Raw Cell 은 Notebook 에서는 아무일도 하지 않지만, `nbconvert` 를 사용하여 다른 파일형식으로 변환 할때 Cell 의 내용을 그대로 넘겨줍니다.
* Toolbox 에서 `Run`을 누르거나 단축키(CTRL + ENTER 또는 SHIFT + ENTER)를 눌러 Cell 을 실행합니다.

In [ ]:
for i in range(3):
    print("Hello World!")

## JupyterLab

* 조금 더 IDE(통합개발환경)스럽게!
* https://jupyter.nims.re.kr/user/(id)/lab
* 아직은 pre-beta 단계.

## JupyterHub

> A multi-user version of the notebook designed for companies, classrooms and research labs
> https://jupyter.org

* 각 사용자에게 Configurable Proxy 를 통해 Jupyter Notebook Server 를 연결해 주는 방식으로 작동합니다.

## Jupyter Notebook, Lab, Hub (왜이리 복잡해???)

* 가장 핵심적인 기능만을 가지고 있는 가벼운 환경을 제공하고, 나머지 기능들은 필요에따라 사용자가 설치하도록 하는 모듈화를 지향합니다.

# NIMS JupyterHub 의 한계

* 여러사람에게 동일한 환경을 제공하기 위한 설계
  * 한정된 자원을 공유
  * 하드웨어 확장이 불편
  * 개인별 컨테이너 제공으로 독립된 환경을 제공 하지만, 정해진 시간(매주 월요일 새벽 1시)에 초기화
  

# Tutorial의 목표

* AWS EC2 를 통해 내게 필요한 만큼의 자원을 할당하여,
* 미리 준비된 Docker Image 를 통해
* 독립적이면서도 규격화된 Jupyter Notebook 환경을 구성.
  
  
* 난이도 : 초급
* 환경설정에 많은 시간을 투자하기 보다는 문제를 해결하는것에 집중 할 수 있도록 Cloud 자원과 Docker Image 를 바로 활용 하는 방법을 설명하려고 합니다.

## Scenario

*실제 사례*

* Machine Learning 을 공부하던 대학원생이 수업중인 서버의 메모리를 모두 소진하여 실습이 10여분간 중단
* 이후 그 학생은 ~~게임용~~ 노트북에 Linux를 설치하고, tensorflow와 jupyter를 설치
* ~~처음 해보는 설치라 이틀간 고생한건 비밀~~
* 작업을 계속하지만, 메모리는 여전히 부족하고 극심한 발열과 소음에 시달림
* 서버관리자(=me)가 AWS 같은 cloud 환경을 추천
* AWS는 뭔가 어렵고 비싸다며 외면
* 웬만한 PC보다 괜찮은 환경을 PC방 요금으로 제공 (`p2.xlarge`, K80 GPU, 61GiB 메모리, 시간당 \$0.9)
* Spot instance 를 사용하면 더 저렴하게 이용가능
* AWS에서 등록된 교육기관의 학생에게 매년 \$100 지원
* ~~광고아님~~

### 다음과 같은 분들에게 추천합니다.

* 공용 서버가 불편하신 분 (한정된 자원, 데이터 손실 위험, 정기점검 등등...)
* PC나 노트북을 그만 혹사 시키고 싶으신 분
* GPU가 설치된 서버나 워크스테이션의 구입을 망설이시는 분
* 대용량 계산이 필요하신분 (`p2.16xlarge` 는 시간당 \$14.4에 732GiB 메모리와 16개의 K80 GPU를 제공)
* Linux kernel, 환경설정, Package 버전 관리 등이 귀찮으신 분
* 시간과 장소에 구애받지 않고 작업하고 싶으신 분

### 목표
* 어렵지 않습니다.
* 다른 사람이 만든 환경을 가져와 사용하는 법을 배웁니다. 따라서 환경설정에 시간을 낭비 할 필요 없습니다.
* 아쉬운 부분이 있으면 살짝 추가하는 법도 배웁니다.
* 완전히 동일한 방법으로 내 PC에도 같은 환경을 구성 할 수 있습니다.

### 진행순서

1. AWS 맛보기
2. Docker 설치와 사용
3. tensorflow-gpu 이미지 사용해보기
4. jupyterlab 추가해보기
5. 마무리

# AWS 맛보기

## 준비

* AWS Account
  * Limits -> Request limit increase
    * Limit Type : EC2 Instances
      * Region : Asia Pacific (Seoul)
      * Primary Instance Type : `p2.xlarge`
      * Limit : Instance Limit
      * New limit value : 1
    * Contact method : Web
  * AWS 에 등록한 e-mail 로 회신. 15분~3일 소요.
  

* SSH client
  * SSH Tunneling을 사용할 수 있는 SSH 클라이언트 프로그램이 필요합니다.
  * Unix 계열 (Linux, macOS, ...) : OpenSSH
    * -i identity_file
    * -L local_port:remote_address:remote_port
    * e.g > `ssh -i jupyter.pem.txt -L 8888:localhost:8888 user@aws.something.amazon.com`
  * MS-Windows : [PuTTY](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html)
    * Connection -> SSH -> Auth -> Private key file for authentication: `jupyter.pem.txt`
    * Connection -> SSH -> Tunnels:
      * Source port : `8888`
      * Destination : `localhost:8888`

### 키 페어 생성
1. 메뉴 -> 네트워크 및 보안 -> 키 페어
![Key Pair](keypair.png)

2. 생성
![Generate Key Pair](keypair_gen.png)

3. 자동으로 다운로드되는 `jupyter.pem.txt` 를 확인합니다.
4. Unix 계열은 파일의 권한을 소유자 읽기전용 (400) 으로 변경합니다.
```bash
chmod 400 jupyter.pem.txt
```

## AWS ECS (EC2 Container Service)

* **장점** : Docker image 만 있으면 끝.
* **단점** : GPU 연산을 위한 `p2` instance 생성 불가.
* 좋아보이긴하지만, 클러스터 관리를 위한 편의 기능 중심으로 당장은 우리의 관심 밖.
* ~~Docker 가 대세는 대세~~


## Eazy way : prepared AMI

* **장점** : AWS 에서 공식적으로 제공하는 미리 준비된 환경. `p2` instance 사용 가능.
* **단점** : 조금은 느린 버전관리. 내 PC에서는 사용할 수 없음.

### 인스턴스 시작
#### 1. 메뉴 -> 인스턴스 -> 인스턴스
![Start Instance](instance_start.png)

#### 2. 빠른시작 -> Deep Learning AMI Ubuntu Version 2.3_Sep2017
![Select Image](instance_image.png)

#### 3. 인스턴스 유형 선택 (`p2.xlarge`)
![p2.xlarge](instance_p2.png)

#### 4. 검토 및 시작 -> 키 페어 선택
![Select Key Pair](instance_keypair.png)

#### 5. 인스턴스 상태가 `running` 이 될때까지 기다림
![Running](instance_running.png)

#### 6. 퍼블릭 DNS 를 확인하고 SSH를 통해 인스턴스에 접속
```bash
ssh -i jupyter.pem.txt -L 8888:localhost:8888 ubuntu@ec2-13-125-8-58.ap-northeast-2.compute.amazonaws.com
```
![SSH](instance_ssh.png)

#### 7. Jupyter Notebook을 실행
```bash
jupyter notebook
```
![Jupyter](instance_jupyter.png)

#### 8. 웹브라우저 주소창에 URL을 복사,붙여넣기

#### 9. 브라우저 창을 끄고, 터미널에서 `cmd+c` 로 jupyter를 종료한 후 로그아웃

#### 10. AWS에서 오른쪽 클릭하여 메뉴에서 인스턴스 종료를 선택
![Terminate](instance_term.png)

# Docker

> Docker is the leading Containers as a Service (CaaS) platform
> https://www.docker.com/what-docker


## 설치

#### 1. Ubuntu Server 16.04 LTS AMI로 `p2.xlarge` 인스턴스를 시작 (스토리지 추가:50GiB)
![Ubuntu](docker_ubuntu.png)
![Storage](docker_storage.png)

#### 2. SSH 로 인스턴스에 접속
```bash
ssh -i jupyter.pem.txt -L 8888:localhsot:8888 ubuntu@PUBLIC_DOMAIN_OF_INSTANCE
```

#### 3. Install Docker
```bash
curl -fsSL get.docker.com | sudo sh

sudo usermod -aG docker ubuntu
```

#### 4. Logout & Login
```bash
exit

ssh -i jupyter.pem.txt -L 8888:localhsot:8888 ubuntu@PUBLIC_DOMAIN_OF_INSTANCE

docker ps
```

## Basic Commands

```bash
docker [OPTIONS] COMMAND [ARGS]
```

### pull
```bash
docker pull [OPTIONS] REPOSITORY_NAME[:TAG|@DIGEST]

docker pull jupyter/tensorflow-notebook:latest
```
![pull](docker_pull.png)

### images
```bash
docker images [OPTIONS] [REPOSITORY[:TAG]]

docker images
```
![images](docker_images.png)

### run
```bash
docker run [OPTIONS] IMAGE [COMMAND] [ARG...]
```
|option|description|
|-|-|
| -p, --publish | Publish a container's port(s) to the host |
| -v, --volume | Bind mount a volume |
| --name | Assign a name to the container |
| --rm | Automatically remove the container when it exits |
| -d, --detach | Run container in background and print container ID |
```bash
mkdir work

sudo chown 1000.100 work/

docker run -it -p 8888:8888 -v /home/ubuntu/work/:/home/jovyan/work/ --name jupyter jupyter/tensorflow-notebook:latest
```
* -d 옵션을 사용하지 않았더라도, `ctrl+p+q`를 사용하여 detach 가능.
![run](docker_run.png)

### ps
```bash
docker ps [OPTIONS]

docker ps
```
![ps](docker_ps.png)

### attach
```bash
docker attach [OPTIONS] CONTAINER

docker attach jupyter
```
* `ctrl+p+q`를 사용하여 detach.

### stop
```bash
docker stop [OPTIONS] CONTAINER [CONTAINER...]

docker stop jupyter
```

### start
```bash
docker start [OPTIONS] CONTAINER [CONTAINER...]


docker ps

docker ps -a

docker start jupyter
```

### logs
```bash
docker logs [OPTIONS] CONTAINER


docker logs jupyter
```
![logs](docker_logs.png)

### rm * 지금은 실행하지 않습니다.
```bash
docker rm [OPTIONS] CONTAINER [CONTAINER...]


# 아직 실행하지 마세요.
# docker stop jupyter

# docker rm jupyter
```

### rmi * 지금은 실행하지 않습니다.
```bash
docker rmi [OPTIONS] IMAGE [IMAGE...]


docker images

# 아직 실행하지 마세요.
# docker rmi jupyter/tensorflow-notebook:latest
```
![rmi](docker_rmi.png)

## Simple NN Test (without GPU)

```bash
docker logs jupyter
```

* 웹 브라우저에서 jupyter notebook 으로 접속
* http://localhost:8888/lab 으로 접속
* `work/` 디렉토리로 이동
* `Simple_NN_Test.ipynb` 파일을 업로드
* `Simple_NN_Test.ipynb` 파일을 더블클릭하여 열기
* 각 Cell 을 실행

![without GPU](test_without_gpu.png)

* 탭 종료
* Terminal 에서 `docker rm -f jupyter` 명령으로 컨테이너 삭제

# Docker with GPU

## Install CUDA
https://developer.nvidia.com/cuda-downloads

```bash
wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.0.176-1_amd64.deb

sudo dpkg -i cuda-repo-ubuntu1604_9.0.176-1_amd64.deb

sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub

sudo apt update

sudo apt install cuda
```
![CUDA](cuda.png)

## Install `nvidia-docker`
https://github.com/NVIDIA/nvidia-docker

```bash
wget https://github.com/NVIDIA/nvidia-docker/releases/download/v1.0.1/nvidia-docker_1.0.1-1_amd64.deb

sudo dpkg -i nvidia-docker_1.0.1-1_amd64.deb

nvidia-docker run --rm nvidia/cuda nvidia-smi
```
![nvidia-docker](nvidia.png)

## Run `tensorflow/tensorflow:latest-gpu-py3`
https://hub.docker.com/r/tensorflow/tensorflow/

```bash
nvidia-docker run -it -p 8888:8888 -v /home/ubuntu/work/:/notebooks/work/ --name tf_gpu tensorflow/tensorflow:latest-gpu-py3
```
![TF with GPU](tf_gpu.png)

* 웹 브라우저에서 jupyter notebook 에 접속
* `work/` 디렉토리로 이동하여 `Simple_NN_Test.ipynb`를 다시 실행
![with GPU](test_with_gpu.png)

## JupyterLab 설치

* http://localhost:8888/lab 으로 들어가 보아도 JupyterLab 이 실행되지 않음. (404 : Not Found)
* JupyterLab 이 설치되어 있는 우리만의 docker image 생성

```bash
mkdir tf_jupyterlab

cd tf_jupyterlab

vi Dockerfile
```

* https://github.com/jupyterlab/jupyterlab 에서 설치방법을 확인

```Dockerfile
FROM tensorflow/tensorflow:latest-gpu-py3

RUN pip install jupyterlab
RUN jupyter serverextension enable --py jupyterlab --sys-prefix
```

* `build` command로 이미지를 생성

```bash
docker build -t tf:gpu-lab-0.1 .

docker images
```

![build](docker_build.png)

```bash
docker stop tf_gpu

nvidia-docker run -it -p 8888:8888 -v /home/ubuntu/work/:/notebooks/work/ --name tf_lab tf:gpu-lab-0.1
```

# 마무리

* 인스턴스의 /home/ubuntu/work/ 디렉토리를 backup. 로컬 머신에서 
```bash
scp -i jupyter.pem.txt -r ubuntu@PUBLIC_DOMAIN_OF_INSTANCE:~/work/ ./
```
* 인스턴스를 종료

# Docker with GPU (Summary)

## p2.xlarge instance + nvidia-docker + tensorflow-gpu image

1. Launch Ubuntu 16.04 instance.
[`p2.xlarge`](https://aws.amazon.com/ko/ec2/instance-types/p2/) with more storage (50GiB).
1. Connect into the instance with SSH tunneling.
```bash
chmod 400 <key.pem.txt>
ssh -i <key.pem.txt> -L 8888:localhsot:8888 ubuntu@<public domain of instance>
sudo apt-get update && sudo apt-get -y upgrade
```
1. Install Docker.
```bash
curl -fsSL get.docker.com | sudo sh
sudo usermod -aG docker ubuntu
```
1. Logout and Login again (for docker group).
```bash
exit
ssh -i <key.pem.txt> -L 8888:localhsot:8888 ubuntu@<public domain of instance>
docker ps
```
1. Install CUDA.
https://developer.nvidia.com/cuda-downloads
```bash
wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
sudo dpkg -i cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
sudo apt update && sudo apt -y install cuda
```
1. Install `nvidia-docker`.
https://github.com/NVIDIA/nvidia-docker
```bash
wget https://github.com/NVIDIA/nvidia-docker/releases/download/v1.0.1/nvidia-docker_1.0.1-1_amd64.deb
sudo dpkg -i nvidia-docker_1.0.1-1_amd64.deb
```
1. Test `nvidia-smi`.
```bash
# docker pull nvidia/cuda:latest
nvidia-docker run --rm nvidia/cuda nvidia-smi
```
1. Run `tensorflow/tensorflow:latest-gpu-py3`.
https://hub.docker.com/r/tensorflow/tensorflow/
```bash
mkdir work/
sudo chown 1000.100 work/
# docker pull tensorflow/tensorflow:latest-gpu-py3
nvidia-docker run -it -p 8888:8888 -v /home/ubuntu/work/:/notebooks/work/ --name tf_gpu tensorflow/tensorflow:latest-gpu-py3
```
1. **Important!!** Backup your `work/` directory!!!